In [1]:
!pip install transformers==4.28.0 sentencepiece sacremoses datasets sacrebleu GPUtil

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 56.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 13.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=21eca579fe520ab64030901835dfb64e60818049694cd6a66a0340abe0985603
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7409 sha256=5aa45fffdf82347a60a68e8f8c3186db4c8a0c11ec8b00790afc30ecf3330e97
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
Successfully built sacremoses GPUtil
  Attempting uninstall: transformers
    Found existing installation: transformers 4.30.1
    Uninsta

In [2]:
import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache() 

Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 |  4% |  2% |


In [3]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
import wandb
user_secrets = UserSecretsClient()
secret_value = user_secrets.get_secret("HF_KEY")
login(secret_value)
secret_value1 = user_secrets.get_secret("WANDB_KEY")
wandb.login(key=secret_value1)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
from datasets import load_dataset, Dataset
training_data = load_dataset("ethansimrm/wmt_16_19_22_biomed_train_processed", split = "train") 
validation_data = load_dataset("ethansimrm/wmt_20_21_biomed_validation", split = "validation")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/ethansimrm--wmt_16_19_22_biomed_train_processed-d78929ff2f62bc10/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8. Subsequent calls will reuse this data.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/ethansimrm--wmt_20_21_biomed_validation-727b0ab23cf9429a/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8. Subsequent calls will reuse this data.


In [5]:
from datasets import load_metric
metric = load_metric("sacrebleu")

In [6]:
#Converts from SRC [TAB] TGT [NEWLINE] to the usual dictionary format
def convertToDictFormat(data): #input should be of the form data['text'], etc.
    source = []
    target = []
    for example in data:
        example = example.strip()
        sentences = example.split("\t")
        source.append(sentences[0])
        target.append(sentences[1])
    ready = Dataset.from_dict({"en":source, "fr":target})
    return ready

In [7]:
train_data_ready = convertToDictFormat(training_data['text'])
val_data_ready = convertToDictFormat(validation_data['text'])

In [8]:
#Load correct tokenizer
from transformers import AutoTokenizer
checkpoint = "Helsinki-NLP/opus-mt-en-fr"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [9]:
source_lang = 'en'
target_lang = 'fr'
def preprocess_function(examples):
    inputs = examples[source_lang]
    targets = examples[target_lang]
    model_inputs = tokenizer(inputs, text_target=targets, padding="longest")
    #Pad to longest sequence in batch, no truncation - we filtered too-long sentences out already
    return model_inputs

In [10]:
tokenized_train = train_data_ready.map(preprocess_function, batched=True)
tokenized_val = val_data_ready.map(preprocess_function, batched=True)

  0%|          | 0/650 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [11]:
#Builds batches from dataset
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [12]:
import numpy as np

def postprocess_text(preds, labels): 
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True) #Convert back into words

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id) #Ignore padded labels added by the data collator to the test set
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True) 

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels) #Remove leading and trailing spaces

    result = metric.compute(predictions=decoded_preds, references=decoded_labels) #BLEU score for provided input and references
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens) #Compute mean prediction length
    result = {k: round(v, 4) for k, v in result.items()} #Round score to 4dp
    return result

In [13]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [14]:
from transformers import EarlyStoppingCallback, IntervalStrategy
import torch

batch_size = 16 #Set as high as possible per Popel & Bojar (2018)

training_args = Seq2SeqTrainingArguments( #Collects hyperparameters
    output_dir="opus_wmt_finetuned_enfr",
    evaluation_strategy=IntervalStrategy.STEPS, #Evaluates every N steps
    save_steps=8000, #Save model every evaluation
    eval_steps=8000, #Evaluate every ~128k sentences
    num_train_epochs=3, #About 1hr per eval interval,5 eval intervals per epoch, 12hr compute budget
    learning_rate=2e-5, #Initial learning rate for AdamW
    per_device_train_batch_size=batch_size, 
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01, #Weight decay for loss computation; Loss = Loss + WD * sum (weights squared)
    save_total_limit=3, #Number of checkpoints to save
    predict_with_generate=True, #Use with ROUGE/BLEU and other translation metrics (see below)
    fp16=True, #Remove fp16 = True if not using CUDA
    push_to_hub=True,
    metric_for_best_model='bleu', #Determines our best model
    load_best_model_at_end=True, #We will choose the best model from among our 5 checkpoints
)

trainer = Seq2SeqTrainer( #Saves us from writing our own training loops
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=0.1)]
    #If we don't improve by at least 0.1 BLEU on the validation set for 3 evaluations, we stop and find the best 
    #model of the 5 we have saved
)

Cloning https://huggingface.co/ethansimrm/opus_wmt_finetuned_enfr into local empty directory.


Download file pytorch_model.bin:   0%|          | 1.40k/285M [00:00<?, ?B/s]

Download file runs/Jun19_16-53-26_52f716f83ae7/events.out.tfevents.1687193616.52f716f83ae7.23.0: 100%|########…

Download file runs/Jun19_19-35-50_9eb095590415/events.out.tfevents.1687203446.9eb095590415.22.0: 100%|########…

Download file runs/Jun19_19-35-50_9eb095590415/1687203446.305718/events.out.tfevents.1687203446.9eb095590415.2…

...

Download file training_args.bin: 100%|##########| 3.68k/3.68k [00:00<?, ?B/s]

Clean file training_args.bin:  27%|##7       | 1.00k/3.68k [00:00<?, ?B/s]

Clean file source.spm:   0%|          | 1.00k/760k [00:00<?, ?B/s]

Clean file target.spm:   0%|          | 1.00k/784k [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/285M [00:00<?, ?B/s]

In [15]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Currently logged in as: ethansimrm. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.15.4
wandb: Run data is saved locally in /kaggle/working/wandb/run-20230620_204341-ihlr8q3q
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run helpful-sound-22
wandb: ⭐️ View project at https://wandb.ai/ethansimrm/huggingface
wandb: 🚀 View run at https://wandb.ai/ethansimrm/huggingface/runs/ihlr8q3q


Step,Training Loss,Validation Loss,Bleu,Gen Len
8000,0.173300,0.109917,40.009500,34.286700
16000,0.167700,0.107222,41.072800,34.884900
24000,0.158300,0.106187,40.859900,34.197500
32000,0.153200,0.104782,41.659600,34.523500
40000,0.152900,0.104135,41.812900,35.269400
48000,0.137400,0.104340,41.133000,36.247400
56000,0.143700,0.103476,39.168100,37.840800
64000,0.140500,0.103072,42.205900,34.122700
72000,0.140100,0.102553,42.165700,35.377800
80000,0.140500,0.102454,42.764500,33.961600


TrainOutput(global_step=104000, training_loss=0.14914121459080623, metrics={'train_runtime': 37357.2448, 'train_samples_per_second': 52.134, 'train_steps_per_second': 3.258, 'total_flos': 8.159003386734182e+16, 'train_loss': 0.14914121459080623, 'epoch': 2.56})